In [1]:
import os
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\Chris Partee\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [2]:
mars_data = {}

In [4]:
# # Enter whatever URL you like
browser.visit("https://redplanetscience.com/")

# Add the page source to the variable `content`.
content = browser.html
# Load the contents of the page, its source, into BeautifulSoup 

soup = BeautifulSoup(content,features="lxml")
# print (soup.prettify)

In [5]:
# Object is “results”, brackets make the object an empty list.
# We will be storing our data here.
results = []
for element in soup.findAll(attrs={'class': 'list_text'}):
    news={}
    title = element.find(attrs={'class':'content_title'})
#     print (title.text)
    para = element.find(attrs={'class':'article_teaser_body'})
#     print (para.text)
    news['news_title']=title.text
    news['news_p']=para.text

    results.append(news)

mars_data["news"] = results
print(results)

[{'news_title': 'Mars Scientists Investigate Ancient Life in Australia', 'news_p': "Teams with NASA's Mars 2020 and ESA's ExoMars practiced hunting for fossilized microbial life in the Australian Outback in preparation for their Red Planet missions. "}, {'news_title': 'The MarCO Mission Comes to an End', 'news_p': 'The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.'}, {'news_title': "Celebrate Mars Reconnaissance Orbiter's Views From Above", 'news_p': 'Marking its 15th anniversary since launch, one of the oldest spacecraft at the Red Planet has provided glimpses of dust devils, avalanches, and more.'}, {'news_title': "The Detective Aboard NASA's Perseverance Rover", 'news_p': 'An instrument called SHERLOC will, with the help of its partner WATSON, hunt for signs of ancient life by detecting organic molecules and minerals.'}, {'news_title': "From JPL's Mailroom to Mars and Beyond", 'news_p': 'Bill Allen has thrived as the mechanical systems design le

In [7]:
browser.visit("https://spaceimages-mars.com/")

# Add the page source to the variable `content`.
content = browser.html
# Load the contents of the page, its source, into BeautifulSoup 
# class, which analyzes the HTML as a nested data structure and allows to select
# its elements by using various selectors.
soup = BeautifulSoup(content,features="lxml")
print(soup.prettify)

<bound method Tag.prettify of <html class=""><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:17px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-

In [11]:
imgtag = soup.find(attrs={'class':'headerimage fade-in'})
featured_image_url = imgtag['src']
featured_image_url = f"https://spaceimages-mars.com/{featured_image_url}"

'https://spaceimages-mars.com/image/featured/mars1.jpg'

In [ ]:
mars_data["space_img"] = featured_image_url

In [ ]:
#pulling table using pandas to scrap
#then exporting to be used later in flask
tables = pd.read_html("https://galaxyfacts-mars.com")
mars_facts = tables[1]
mars_facts.columns = ["Data Type", "Info"]
mars_facts.drop(index=mars_facts.index[0], axis=0,inplace=True)
mars_facts = mars_facts.set_index("Data Type")

In [ ]:
mars_data["facts_table"] = mars_facts.to_html()

In [ ]:
#connecting to the hemisphere page
browser.visit("https://marshemispheres.com/")
content = browser.html

soup = BeautifulSoup(content,features="lxml")

In [ ]:
#grabbing the headers text of images on home page that have links
mars_hemispheres = []

for x in soup.findAll("div", attrs={"class": "item"}):
    ltext = x.find("h3")
    if ltext:
        mars_hemispheres.append(ltext.text)


In [ ]:
#making connection with browser to run the click loop
browser.visit("https://marshemispheres.com/")

In [ ]:
#click loop that finds the name and url of the original images
hemis_imgs = []

for m in mars_hemispheres:
    hemis = {}
    
    browser.find_by_text(m).first.click()
    visit = browser.links.find_by_partial_href("full.jpg").first
    hemis["tif_url"] = visit["href"]
    hemis["title"] = browser.find_by_css("h2.title").text
    
    hemis_imgs.append(hemis)
    
    browser.back()
    
    
browser.quit()

In [ ]:
mars_data["hemi_pics"] = hemis_imgs
timestamp = dt.datetime.now()
mars_data["last_modified"] = timestamp